In [36]:
from zope.interface import Interface, implementer
from zope.interface.verify import verifyClass
import attr
import sys
import pyrsistent
from pyrsistent import m, v
from __future__ import annotations
import pprint

```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\title{Why Change?}
\subtitle{Life is Better Without It}
\author{Moshe Zadka -- https://cobordism.com}
\date{2020}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}[fragile]
```

In [8]:
class IRectangle(Interface):
    def get_height() -> float:
        """Return height"""
    def get_width() -> float:
        """Return width"""
    def set_height(height: float):
        """Set height"""
    def set_width(width: float):
        """Set width"""

```{=latex}
\end{frame}
```

In [9]:
@implementer(IRectangle)
@attr.s(auto_attribs=True)
class Square:
    side: float

    def get_height(self) -> float:
        return side
    
    def get_width(self) -> float:
        return side
try:
    verifyClass(IRectangle, Square)
except Exception as exc:
    print(str(exc).split(":")[1], file=sys.stderr)


    The __main__.IRectangle.set_height(height) attribute was not provided
    The __main__.IRectangle.set_width(width) attribute was not provided


In [10]:
@implementer(IRectangle)
@attr.s(auto_attribs=True)
class Square:
    _side: float

    def get_height(self) -> float:
        return self._side
    
    def get_width(self) -> float:
        return self._side

    def set_height(self, height: float):
        self._side = height # ???
        
    def set_width(self, width: float):
        self._side = width # ???

In [11]:
def area(rectangle: IRectangle) -> float:
    return rectangle.get_height() * rectangle.get_width()

def double_height(rectangle: IRectangle) -> float:
    rectangle.set_height(2 * rectangle.get_height())

x = Square(side=5)
print(area(x))
double_height(x)
print(area(x))

25
100


In [12]:
class IRectangle(Interface):
    def get_height() -> float:
        """Return height"""
    def get_width() -> float:
        """Return width"""
    def with_height(height: float) -> IRectangle:
        """Rectangle with same width, new height"""
    def with_width(width: float) -> IRectangle:
        """Rectangle with same height, new width"""

In [13]:
@implementer(IRectangle)
@attr.s(auto_attribs=True, frozen=True)
class Rectangle:
    _height: float
    _width: float
    
    def get_height(self) -> float:
        return self._height
    
    def get_width(self) -> float:
        return self._width

    def with_height(self, height) -> float:
        return attr.evolve(self, height=height)
    
    def with_width(self, width) -> float:
        return attr.evolve(self, width=width)

In [14]:
@implementer(IRectangle)
@attr.s(auto_attribs=True)
class Square:
    _side: float

    def get_height(self) -> float:
        return self._side
    
    def get_width(self) -> float:
        return self._side

    def with_height(self, height: float) -> IRectangle:
        return Rectangle(width=self._side, height=height)
    
    def with_width(self, width: float) -> IRectangle:
        return Rectangle(height=self._side, width=width)

try:
    verifyClass(IRectangle, Square)
except Exception as exc:
    print(str(exc).split(":")[1], file=sys.stderr)

In [15]:
def area(rectangle):
    return rectangle.get_height() * rectangle.get_width()

def double_height(rectangle):
    return rectangle.with_height(2 * rectangle.get_height())

x = Square(side=5)
print(area(x))
print(area(double_height(x)))

25
50


In [16]:
def sum_with_extra(e1, e2, things=[]):
    things.append(e1)
    things.append(e2)
    return sum(things)

In [17]:
sum_with_extra(1, 2, [3, 4])

10

In [18]:
sum_with_extra(1, 2)

3

In [19]:
# Whoops!
sum_with_extra(1, 2)

6

In [20]:
def sum_with_extra_v2(e1, e2, things=None):
    if things is None:
        things = []
    things.append(e1)
    things.append(e2)
    return sum(things)

In [21]:
sum_with_extra_v2(1, 2, [3, 4])

10

In [22]:
sum_with_extra_v2(1, 2)

3

In [23]:
sum_with_extra_v2(1, 2)

3

In [24]:
things = [1, 2]
sum_with_extra_v2(1, 2, things)

6

In [25]:
sum_with_extra_v2(1, 2, things)

9

In [26]:
def sum_with_extra_v3(e1, e2, things=None):
    if things is None:
        things = []
    things = things.copy()
    things.append(e1)
    things.append(e2)
    return sum(things)

In [27]:
from pyrsistent import v

In [28]:
def sum_with_extra_p_v1(e1, e2, things=v()):
    things = things.append(e1)
    things = things.append(e2)
    return sum(things)

In [29]:
sum_with_extra_p_v1(1, 2)

3

In [30]:
sum_with_extra_p_v1(1, 2)

3

In [31]:
things = v(1, 2)
sum_with_extra_p_v1(1, 2, things)

6

In [32]:
sum_with_extra_p_v1(1, 2, things)

6

In [34]:
stats = m(
    frontend=m(
        web_1=m(hits=53),
        web_2=m(hits=78)),
    backend=m(
        db1=m(queries=23),
        db2=m(queries=11)))

In [37]:
new_stats = stats.transform(v("frontend", "web_1", "hits"), lambda x: x + 1)
pprint.pprint(pyrsistent.thaw(new_stats))

{'backend': {'db1': {'queries': 23}, 'db2': {'queries': 11}},
 'frontend': {'web_1': {'hits': 54}, 'web_2': {'hits': 78}}}


```{=latex}
\end{document}
```